In [1]:
!pip install gymnasium[atari] gymnasium[accept-rom-license] ale-py autorom


In [2]:
!AutoROM --accept-license


AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	C:\Users\thoma\anaconda3\Lib\site-packages\AutoROM\roms

Existing ROMs will be overwritten.


In [3]:
import ale_py
import gymnasium as gym

env = gym.make("ALE/Tennis-v5")
env


<OrderEnforcing<PassiveEnvChecker<AtariEnv<ALE/Tennis-v5>>>>

In [4]:
import gymnasium as gym
gym.make("ALE/Tennis-v5")


<OrderEnforcing<PassiveEnvChecker<AtariEnv<ALE/Tennis-v5>>>>

In [5]:
import gymnasium as gym
import numpy as np
import time
import matplotlib.pyplot as plt

from stable_baselines3 import DQN, PPO, A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.atari_wrappers import AtariWrapper


In [6]:
def make_env():
    env = gym.make("ALE/Tennis-v5", frameskip=1)
    env = AtariWrapper(env)
    env = Monitor(env)
    return env

env = DummyVecEnv([make_env])
env = VecFrameStack(env, n_stack=4)


In [ ]:
start = time.time()

dqn = DQN(
    "CnnPolicy",
    env,
    learning_rate=1e-4,
    buffer_size=100000,
    learning_starts=10000,
    batch_size=32,
    gamma=0.99,
    train_freq=4,
    target_update_interval=1000,
    verbose=1
)

dqn.learn(total_timesteps=300000)

dqn_time = time.time() - start
dqn_reward, _ = evaluate_policy(dqn, env, n_eval_episodes=10)

dqn_reward, dqn_time


Using cpu device
Wrapping the env in a VecTransposeImage.


c:\Users\thoma\anaconda3\Lib\site-packages\stable_baselines3\common\buffers.py:242: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 5.65GB > 1.01GB
  warnings.warn(


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.62e+03 |
|    ep_rew_mean      | -24      |
|    exploration_rate | 0.795    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1004     |
|    time_elapsed     | 6        |
|    total_timesteps  | 6484     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.62e+03 |
|    ep_rew_mean      | -24      |
|    exploration_rate | 0.589    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 331      |
|    time_elapsed     | 39       |
|    total_timesteps  | 12986    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000176 |
|    n_updates        | 746      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean    

In [ ]:
start = time.time()

ppo = PPO(
    "CnnPolicy",
    env,
    learning_rate=2.5e-4,
    n_steps=128,
    batch_size=64,
    gamma=0.99,
    verbose=1
)

ppo.learn(total_timesteps=300000)

ppo_time = time.time() - start
ppo_reward, _ = evaluate_policy(ppo, env, n_eval_episodes=10)

ppo_reward, ppo_time


In [ ]:
start = time.time()

a2c = A2C(
    "CnnPolicy",
    env,
    learning_rate=7e-4,
    gamma=0.99,
    verbose=1
)

a2c.learn(total_timesteps=300000)

a2c_time = time.time() - start
a2c_reward, _ = evaluate_policy(a2c, env, n_eval_episodes=10)

a2c_reward, a2c_time


In [ ]:
algos = ["DQN", "PPO", "A2C"]
scores = [dqn_reward, ppo_reward, a2c_reward]
times = [dqn_time, ppo_time, a2c_time]

plt.figure()
plt.bar(algos, scores)
plt.title("Score moyen par algorithme — Tennis Atari")
plt.ylabel("Score moyen")
plt.show()

for a, s, t in zip(algos, scores, times):
    print(a, "Score moyen:", s, "Temps:", t)
